In [13]:
import transformers
import torch, math, einops
from src import Tokenizer, linear_unidirectional_graph_maker

In [14]:
tokenizer = Tokenizer('gpt2')
pretraied = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [44]:
from src.GPT2 import GPT2, GPT2_Encoder, GPT2_LM_Head

encoder=GPT2_Encoder()
decoder=GPT2_LM_Head()

model=GPT2(encoder, decoder, tokenizer,dropout=0)

model.load_from_original(pretraied)

In [16]:
def split_heads(tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

In [17]:
x=torch.randint(0, 50257, (1, 10))
x=torch.randn([1,13,768])

sequence_lenght=17
batch_size=1
n_heads=13
d_Embedding=64*n_heads
Q=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
K=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
V=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])


out, att = pretraied.transformer.h[0].attn._attn(Q,K,V,None,None)
#out=out.permute(1, 0, 2).contiguous().view(batch_size, sequence_lenght, d_Embedding)
out.shape,att.shape
out=out.permute(0,2,1,3).view(sequence_lenght,n_heads,d_Embedding//n_heads)



In [18]:
att_pretrained = einops.einsum(Q,K,'... s e, ... t e -> ... s t')/math.sqrt(K.size(-1))
alignments=att_pretrained
query_length, key_length = Q.size(-2), K.size(-2)

causal_mask = pretraied.transformer.h[0].attn.bias[:, :, key_length - query_length : key_length, :key_length].to(torch.bool)
mask_value = -torch.tensor(float('inf'))
att_pretrained = torch.where(causal_mask, att_pretrained, mask_value)
alignments=att_pretrained
att_pretrained = torch.nn.functional.softmax(att_pretrained, dim=-1)
att_pretrained = att_pretrained.view(n_heads,sequence_lenght,sequence_lenght)

(att_pretrained==att).all()



tensor(True)

In [19]:
from src.transformerMP import attention_message

graph_maker=linear_unidirectional_graph_maker(40)

edge_index=graph_maker(sequence_lenght)
senders, receivers = edge_index

Q1=Q.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
K1=K.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
V1=V.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)

#o,a=attention_message(Q,K,V,edge_index,0)
#o.view(sequence_lenght,d_Embedding)

N=20

i,j=receivers[N],senders[N]
print((Q1[receivers]*K1[senders]).sum(dim=-1)[N]/math.sqrt(K.size(-1)))

print(alignments.view(n_heads,sequence_lenght,sequence_lenght)[:,i,j])

tensor([ 0.4194,  0.1568, -0.0189,  1.6448, -0.9277, -0.5320,  0.4555, -0.7678,
         0.9130, -0.1805,  0.0130, -0.5781, -1.9383])
tensor([ 0.4194,  0.1568, -0.0189,  1.6448, -0.9277, -0.5320,  0.4555, -0.7678,
         0.9130, -0.1805,  0.0130, -0.5781, -1.9383])


In [20]:
from src.transformerMP import normalize_strength
from math import sqrt
def attention_message(Q:torch.Tensor,
                      K:torch.Tensor,
                      V:torch.Tensor,
                      edge_index:torch.Tensor,
                      att_dropout=0.1
                      ):

    senders, receivers = edge_index
    N,h,d=K.shape   

    #Q.K^T 
    att=(Q[receivers]*K[senders]).sum(dim=-1)/sqrt(d)

    #softmax    
    att = torch.exp(att+3-att.max()) #could be done in-plase using the function att.exp_() if memory is a bootleneck
    attention = normalize_strength(att, receivers, N, h)

    #Dropout
    #att=attention_dropout(attention, att_dropout)

    #softmax*V
    att = einops.einsum(attention,V[senders],' ... , ... c -> ... c')
    out = torch.zeros_like(V,device=V.device)

    return out.index_add(0,receivers,att), attention #could be done in-place using the function out.index_add_()

In [21]:
out_function,a=attention_message(Q1,K1,V1,edge_index,0)

i=3
print(edge_index[:,receivers==j])
print(att_pretrained[i,j:,j])
print(a[senders==j,i])
asd=True
for j in range(att_pretrained.shape[-1]):
    asd= asd and torch.allclose(att_pretrained[:,j:,j],a.permute(1,0)[:,senders==j],1e-4,1e-4)
asd
    

tensor([[0, 1, 2, 3, 4, 5],
        [5, 5, 5, 5, 5, 5]])
tensor([0.5131, 0.2150, 0.1793, 0.0338, 0.0323, 0.0166, 0.1188, 0.2253, 0.0064,
        0.0565, 0.0360, 0.1415])
tensor([0.5131, 0.2150, 0.1793, 0.0338, 0.0323, 0.0166, 0.1188, 0.2253, 0.0064,
        0.0565, 0.0360, 0.1415])


True

In [22]:
V=V.view(n_heads,sequence_lenght,d_Embedding//n_heads)
att_pretrained.shape,V.shape
print(att_pretrained.shape,V.shape)
AV_pretrained=einops.einsum(att_pretrained,V,'h i j, h j e -> h i j e')
print(AV_pretrained.shape)

if not (AV_pretrained.sum(-2)==torch.matmul(att_pretrained,V)).all(): #true
    print (False)

print(a.shape,V1[senders].shape)
dsa=einops.einsum(a,V1[senders],'..., ... e -> ... e')
dsa.shape==V1[senders].shape==(152, n_heads, d_Embedding//n_heads) #True

N=9
i,j=senders[N],receivers[N]
print(torch.allclose(dsa[N],AV_pretrained[:,j,i],1e-3,1e-3))

torch.Size([13, 17, 17]) torch.Size([13, 17, 64])
torch.Size([13, 17, 17, 64])
torch.Size([153, 13]) torch.Size([153, 13, 64])
True


In [23]:
out_pretrained=AV_pretrained.sum(-2).permute(1,0,2)
dsum=torch.zeros_like(V1).index_add(0,receivers,dsa)

print(out_pretrained.shape,dsum.shape)
print(torch.allclose(out_pretrained,dsum,1e-3,1e-3))
torch.allclose(out,out_function,1e-3,1e-3)
#print((dsa[N]==asd[:,j,i]))

torch.Size([17, 13, 64]) torch.Size([17, 13, 64])
True


True

# aasdasda

In [48]:
sequence_length=13
heads=12
d_Embedding=64*heads

In [59]:
def split_heads(tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

def merge_heads(tensor, num_heads, attn_head_size):
    """
    Merges attn_head_size dim and num_attn_heads dim into hidden_size
    """
    tensor = tensor.permute(0, 2, 1, 3).contiguous()
    new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
    return tensor.view(new_shape)

def original_c_attn(x,i=0):
        query, key, value = pretraied.transformer.h[i].attn.c_attn(x).split(d_Embedding, dim=2)

        dim=d_Embedding//heads
        query = split_heads(query, heads, dim)
        key = split_heads(key, heads, dim)
        value = split_heads(value, heads, dim)

        return query,key,value

In [69]:

x=torch.randn([1,sequence_length,d_Embedding])

edge_index=graph_maker(13)
Qp,Kp,Vp=original_c_attn(x)
Q,K,P=model.transformer_blocks[0].attention_block.make_QKV(x)

In [72]:
Qp.shape

torch.Size([1, 12, 13, 64])

In [78]:
QQ=Qp.view([heads,sequence_length,d_Embedding//heads]).permute(1,0,2)

QQ[:,0,0],Q[:,0,0]



(tensor([-10.1809,   1.7356,   1.2191,  17.4904,  -9.6279,   7.9092,   1.9963,
          -7.3492,   0.7449,  -1.1058,   0.9803,  -8.7451,  -7.9611],
        grad_fn=<SelectBackward0>),
 tensor([-10.1809,   1.7356,   1.2191,  17.4904,  -9.6279,   7.9092,   1.9963,
          -7.3492,   0.7449,  -1.1058,   0.9803,  -8.7451,  -7.9611],
        grad_fn=<SelectBackward0>))